# Revisión de código (Secciones 3bcd)

Funciones a revisar: 
- formar_abc
- delta
- formar_omegas


In [0]:
import cupy as cp

def formar_vectores(mu, Sigma):
  '''
  Calcula las cantidades u = \Sigma^{-1}  \mu y v := \Sigma^{-1} \cdot 1 del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    u (cupy array, escalar): vector dado por \cdot Sigma^-1 \cdot mu (dimension n)
    v (cupy array, escalar): vector dado por Sigma^-1 \cdot 1 (dimension n)
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  # Obtiene vectores de interes
  u = cp.linalg.solve(Sigma, mu)
  u = u.transpose()[0] # correcion de expresion de array
  v = cp.linalg.solve(Sigma, ones_vector)

  return u , v

# Explicación

Las siguientes funciones son parte fundamental para obtener la ecuación de los pesos que minimicen la varianza del portafolio.

# Función a revisar : 
**formar_abc** (Sección 3.b)

Suponemos que tenemos **n** activos, siendo $\hat{\mu} = \hat{\mu_{1}}, \hat{\mu_{2}}, .. \hat{\mu_{n}}$ el vector de rentornos esperados estimados y  $\hat{\sum_{nxn}}$ la matriz de varianzas y covarianzas esperada de dichos retornos.

Dados los valores de $\hat{\mu}$ y $\hat{\sum_{nxn}}$, la siguiente función busca obtener tres valores denominados **A, B, C** donde:

$$A := \hat{\mu}^T\hat{\Sigma}^{-1}\hat{\mu}$$

$$B := 1^T\hat{\Sigma}^{-1}  1$$

$$C := 1^T\hat{\Sigma}^{-1}\hat{\mu}$$

Dada la función de la sección 3.a denominada **formar_vectores** donde: 
$$ u := \hat{\Sigma}^{-1} \cdot \hat{\mu} $$

$$ v := \hat{\Sigma}^{-1} \cdot 1 $$

La solución para obtener **A,B,C** es la siguiente:

$$A := \hat{\mu}^T u$$

$$B := 1^Tv$$

$$C := 1^Tu$$

**Nota**: los valores de A,B y C son escalares

In [0]:
def formar_abc(mu, Sigma):
  '''
  Calcula las cantidades A, B y C del diagrama de flujo del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  u, v = formar_vectores(mu, Sigma)

  # Obtiene escalares de interes
  A = mu.transpose()@u
  B = ones_vector.transpose()@v
  C = ones_vector.transpose()@u

  return A, B, C

**1. Documentación**
- La documentación expresa de manera concisa y breve el objetivo del código
- Los argumentos de entrada son explicados y detallados correctamente mencionando sus dimensiones correspondientes.
- Los argumentos de salida son expresados correctamente mostrando sus dimensiones y significados.


**2. Cumplimiento de objetivos del código**
- La función cumple con el objetivo devolviendo los escalares A,B y C que serán requeridos posteriormente. 


**3. Tests**

**Objetivo 1**: Desempeño del código alterando el número de activos, es decir **n**

1. n = $10^2$

In [4]:
n= 10**2
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)



A =  [2.02706907]
B =  1.9141406813363295
C =  2.8218915878694695


2. n = $10^3$

In [13]:
n= 10**3
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)

A =  [-1635.3651061]
B =  -1.225027905896697
C =  123.5389049531505


3. n = $10^4$

In [14]:
n= 10**4
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)

A =  [-66.48767904]
B =  1.9973144998778083
C =  1.0972820249845583


4. n = $10^5$

In [15]:
n= 10**5
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)

OutOfMemoryError: ignored

**Objetivo 2:** Operaciones con A,B y C


In [18]:
n= 10**4
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)

print("A*B*C: ", A*B*C)
print("A*B/C: ", A*B/C)
print("A+B+C: ", A+B+C)

A =  [-39.80508806]
B =  1.9681339127906536
C =  1.829163084595848
A*B*C:  [-143.29982559]
A*B/C:  [-42.82928317]
A+B+C:  [-36.00779107]


**Objetivo 3:** Desempeño con $\hat{\mu}_{i}$ de diferentes tamaños

In [37]:
n= 10**3
rt = 10**18 #Tamaño del retorno esperado
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)


A =  [-5.05578153e+36]
B =  1.9888812380119054
C =  3.6368303906023014e+17


In [38]:
n= 10**3
rt = 10**19 #Tamaño del retorno esperado
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)

A =  [-1.13056086e+38]
B =  1.9774140297964762
C =  4.795454397622616e+18


In [39]:
n= 10**4
rt = 10**19 #Tamaño del retorno esperado
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)

A =  [1.03718642e+39]
B =  1.9864997703065628
C =  1.6106599823831794e+19


In [40]:
n= 10**4
rt = 10**20 #Tamaño del retorno esperado
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)

OverflowError: ignored

In [43]:
n= 10**4
rt = -10**18 #Tamaño del retorno esperado
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
A,B,C = formar_abc(mu, Sigma)
print("A = ",A)
print("B = ",B)
print("C = ",C)

A =  [1.63629331e+37]
B =  1.9808794790036426
C =  -5.317820939355341e+17


# Hallazgos

1. La función es posible con un máximo de número de activos **n** de $10^4$, lo cuál esta relacionado con el tamaño máximo de $\Sigma$ de ${10^4x10^4}$
2. La función devuelve arrays con los cuales es posible realizar operaciones
3. La función admite $\hat{\mu}_{i}$ de tamaño igual o menor a $10^{19}$ así como retornos esperados negativos.

# Función a revisar : 
**delta** (Sección 3.c)

Una vez obtenidos los valores A,B y C de la sección 3.b,
se obtiene el valor $\Delta$, donde:

$$\Delta = AB - C^2$$


In [0]:
def delta(A,B,C):
  '''
  Calcula las cantidad Delta = AB-C^2 del diagrama de flujo del problema de Markowitz

  Args:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu

  Return:
    Delta (cupy array, escalar): escalar dado \mu^t \cdot \Sigma^{-1} \cdot \mu
  '''
  Delta = A*B-C**2

  return Delta

**1. Documentación**
- La definición es correcta, breve y concisa
- Los argumentos de entrada son definidos correctamente mostrando el tipo de argumento.
- El argumento de salida es expresado correctamente mostrando el tipo y definición de este.


**2. Cumplimiento de objetivos del código**

El código devuelve un escalar el cual es denominado $\Delta$ cumpliendo con su objetivo.

**3. Tests**

Anteriormente se observó que el número máximo de activos **n** debe ser de tamaño $10^4$ y $\mu_{i}$ de tamaño máximo de $10^{18}$, el siguiente test es para comprobar que este siga siendo el número máximo y no un tamaño menor.


In [45]:
n= 10**4
rt = 10**18 #Tamaño del retorno esperado
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
A,B,C = formar_abc(mu, Sigma)
d = delta(A,B,C)
print("Delta: ",d)


Delta:  [1.3425898e+38]


In [46]:
n= 10**4
rt = -10**18 #Tamaño del retorno esperado
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
A,B,C = formar_abc(mu, Sigma)
d = delta(A,B,C)
print("Delta: ",d)


Delta:  [-7.6719715e+37]


# Hallazgos

El código obtiene el valor de $\Delta$ adecuadamente para un número máximo de retornos de tamaño $10^4$ y para valores de $\hat{\mu_{i}}$ de tamaño máximo $10^{18}$, así como para $\hat{\mu_{i}}$ con valores negativos.


# Función a revisar:

**formar_omegas** (Sección 3.d)

En esta sección se obtendran dos ecalares auxiliares $w_{0}$ y $w_{1}$ denominados multiplicadores de lagrange, para encontrar los pesos que minimicen la varianza del portafolio dado cierto retorno $\hat{r}$ esperado por el inversionista, de la siguiente manera:

$$w_{0} = \frac{1}{\Delta}(\hat{r} \cdot B - C)$$

$$w_{1} = \frac{1}{\Delta}(A - C \cdot \hat{r} )$$

Se hará uso de las funciones anteriores de esta misma sección.



In [0]:
def formar_omegas(r, mu, Sigma):
  '''
  Calcula las cantidades w_o y w_1 del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    w_0 (cupy array, matriz): matriz dada por 
          w_0 = \frac{1}{\Delta} (B \Sigma^{-1} \hat{\mu}- C\Sigma^{-1} 1) 
    w_1 (cupy array, vector): vector dado por 
         w_1 = \frac{1}{\Delta} (C \Sigma^{-1} \hat{\mu}- A\Sigma^{-1} 1)
  '''
  # Obtenemos u = Sigma^{-1} \hat{\mu}, v = \Sigma^{-1} 1
  u, v = formar_vectores(mu, Sigma)
  # Escalares relevantes
  A, B, C = formar_abc(mu, Sigma)
  Delta = delta(A,B,C)
  # Formamos w_0 y w_1
  w_0 = (1/Delta)*(r*B-C)
  w_1 = (1/Delta)*(A-C*r)

  return w_0, w_1

**1. Documentación**
- La definición es correcta, breve y concisa
- Los argumentos de entrada $\hat{\mu}$ y $\hat{\Sigma}$ son definidos correctamente mostrando el tipo de argumento. Hace **falta** detallar la entrada del argumento $\hat{r}$
- Los argumentos de salida son definidos correctamente, sin embargo son **escalares** no matrices

**Sugerencia:** 

argm entrada faltante:

- r (cupy array, escalar): escalar que denota el retorno esperado por el inversionista

argm salida:

- w_0 (cupy array, escalar): escalar dada por $w_0 = \frac{1}{\Delta} (B \Sigma^{-1} \hat{\mu}- C\Sigma^{-1} 1) $
- w_1 (cupy array, escalar): escalar dado por  $w_1 = \frac{1}{\Delta} (C \Sigma^{-1} \hat{\mu}- A\Sigma^{-1} 1)$


**2. Cumplimiento de objetivos del código**

El código devuelve dos matrices $w_{0}$ y $w_{1}$ cumpliendo con el objetivo de la función.

**3. Tests**

**Objetivo 1:**

Anteriormente se observó que el número máximo de activos **n** debe ser de tamaño $10^4$ y los valores de $\hat{\mu_{i}}$ de tamaño máximo de $10^{18}$, el siguiente test es para comprobar que este siga siendo el número máximo y no un tamaño menor.


In [10]:
n= 10**4
r= 10 #retorno esperado del inversionista
rt = 10**18 #Tamaño de mu_i
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
# multiplicadores de lagrange
w0,w1 = formar_omegas(r,mu,Sigma)
print("w0= ", w0)
print("w1= ", w1)

w0=  [-1.53296599e-21]
w1=  [0.49726763]


**Objetivo 2:** Probar el código con tamaños grandes del retorno esperado por el inversionista

1. $\hat{r} = 10^{18}$

In [12]:
n= 10**4
r= 10**18
rt = 10**18 #Tamaño de mu_i
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
# multiplicadores de lagrange
w0,w1 = formar_omegas(r,mu,Sigma)
print("w0= ", w0)
print("w1= ", w1)

w0=  [-4.51189297e-21]
w1=  [0.50847464]


In [13]:
n= 10**4
r= 10**19
rt = 10**18 #Tamaño de mu_i
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
# multiplicadores de lagrange
w0,w1 = formar_omegas(r,mu,Sigma)
print("w0= ", w0)
print("w1= ", w1)

w0=  [-2.52303298e-19]
w1=  [0.59587867]


In [6]:
n= 10**4
r= 10**20
rt = 10**18 #Tamaño de mu_i
Sigma=cp.random.rand(n, n)
mu=cp.random.rand(n, 1)*rt
# multiplicadores de lagrange
w0,w1 = formar_omegas(r,mu,Sigma)
print("w0= ", w0)
print("w1= ", w1)

OverflowError: ignored

# Hallazgos

A continuación se detallan los valores posibles para el vector de retornos esperados $\hat{\mu}$, la matriz de covarianzas $\hat{\Sigma}$, el valor del retorno esperado del portafolio por el inversionista $\hat{r}$  y el número de activos n:

- $\hat{\mu}_{i}$, donde $i = 1,...,n$ valores positivos y negativos cuyo  tamaño debe ser menor o igual a $10^{18}$ 

- $\hat{\Sigma}$ de tamaño máximo: $\hat{\Sigma}_{10^4X10^4}$

- n de tamaño máximo $10^4$, esto se deriva del tamaño máximo de $\hat{\Sigma}$ 


- $\hat{r}$  tamaño máximo $10^{19}$


**Nota**: Los tests fueron realizados en google colaboratory con GPU como entorno de ejecución, en una computadora con las siguientes características:

- MacBook Pro (13-inch, 2019, Four Thunderbolt 3 ports)
- Procesador: 2.4 GHz Intel Core i5 de cuatro núcleos
- Memoria: 8 GB 2133 MHz LPDDR3